In [1]:
# 파일 불러오기
import pandas as pd
import numpy as np

bat_season = pd.read_csv("../data/raw/kbo_batting_stats_by_season_1982-2025.csv")
pit_season = pd.read_csv("../data/raw/kbo_pitching_stats_by_season_1982-2025.csv")

bat_season.head()

,Id,Name,Birthdate,Handedness,School,Draft,Year,Team,Age,Pos.,...,GDP,SH,SF,AVG,OBP,SLG,OPS,R/ePA,wRC+,WAR
0,1001,구천서,1963년 08월 15일,우투우타,옥산초-경운중-신일고,82 OB 창단 멤버,1982,OB,19,2B,...,3,7,0,0.308,0.405,0.421,0.826,0.035,138.9,2.95
1,1001,구천서,1963년 08월 15일,우투우타,옥산초-경운중-신일고,82 OB 창단 멤버,1983,OB,20,2B,...,5,13,2,0.279,0.360,0.406,0.766,0.017,122.9,1.93
2,1001,구천서,1963년 08월 15일,우투우타,옥산초-경운중-신일고,82 OB 창단 멤버,1984,OB,21,2B,...,6,8,3,0.229,0.324,0.279,0.603,-0.023,79.6,0.89
3,1001,구천서,1963년 08월 15일,우투우타,옥산초-경운중-신일고,82 OB 창단 멤버,1985,OB,22,2B,...,3,10,2,0.235,0.354,0.292,0.646,-0.014,88.3,1.72
4,1001,구천서,1963년 08월 15일,우투우타,옥산초-경운중-신일고,82 OB 창단 멤버,1986,OB,23,2B,...,0,1,3,0.205,0.301,0.282,0.583,-0.038,63.1,0.22


In [2]:
# 최신 5개년 추출
bat = bat_season.copy()
bat["Year"] = pd.to_numeric(bat["Year"], errors="coerce")
bat = bat[bat["Year"].between(2021, 2025)].copy()

In [3]:
bat.columns

Index(['Id', 'Name', 'Birthdate', 'Handedness', 'School', 'Draft', 'Year',
       'Team', 'Age', 'Pos.', 'G', 'oWAR', 'dWAR', 'PA', 'ePA', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HP', 'IB', 'SO',
       'GDP', 'SH', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'R/ePA', 'wRC+', 'WAR'],
      dtype='object')

In [4]:
bat = bat.dropna(subset=["Year", "Team"]).copy()
team_year = (
    bat.groupby(["Year", "Team"], as_index=False)
       .agg({
           "PA": "sum",
           "AB": "sum",
           "R": "sum",
           "H": "sum",
           "2B": "sum",
           "3B": "sum",
           "HR": "sum",
           "TB": "sum",
           "RBI": "sum",
           "SB": "sum",
           "CS": "sum",
           "BB": "sum",
           "HP": "sum",   # ✅ HBP가 아니라 HP
           "SO": "sum",
           "GDP": "sum",
           "SH": "sum",
           "SF": "sum",
           "ePA": "sum",  # 있길래 같이 (필요 없으면 빼도 됨)
       })
)

team_year.head()

,Year,Team,PA,AB,R,H,2B,3B,HR,TB,RBI,SB,CS,BB,HP,SO,GDP,SH,SF,ePA
0,2021,KIA,5577,4831,568,1198,183,21,66,1621,546,73,31,595,73,921,119,41,37,5526
1,2021,KT,5626,4810,719,1276,219,11,106,1835,673,112,46,644,58,1047,105,69,44,5541
2,2021,LG,5533,4750,654,1188,210,11,110,1750,621,92,34,592,94,926,102,52,45,5471
3,2021,NC,5530,4806,702,1254,203,17,170,2001,670,101,46,511,111,1093,94,59,43,5460
4,2021,SSG,5735,4898,755,1278,204,13,185,2063,712,100,43,636,94,1055,106,55,52,5672


In [5]:
# 보험
team_year["HP"] = team_year["HP"].fillna(0)
team_year["SF"] = team_year["SF"].fillna(0)

In [6]:
import numpy as np

# (BB+HP)/PA
team_year["BB_HBP_PA"] = (team_year["BB"] + team_year["HP"]) / team_year["PA"].replace(0, np.nan)

# SO/PA, HR/PA
team_year["SO_PA"] = team_year["SO"] / team_year["PA"].replace(0, np.nan)
team_year["HR_PA"] = team_year["HR"] / team_year["PA"].replace(0, np.nan)

# AVG, OBP, SLG는 집계값으로 재계산
team_year["AVG_calc"] = team_year["H"] / team_year["AB"].replace(0, np.nan)

den_obp = (team_year["AB"] + team_year["BB"] + team_year["HP"] + team_year["SF"])
team_year["OBP_calc"] = (team_year["H"] + team_year["BB"] + team_year["HP"]) / den_obp.replace(0, np.nan)

team_year["SLG_calc"] = team_year["TB"] / team_year["AB"].replace(0, np.nan)
team_year["ISO_calc"] = team_year["SLG_calc"] - team_year["AVG_calc"]

team_year[["Year","Team","PA","OBP_calc","ISO_calc","BB_HBP_PA","SO_PA","HR_PA"]].head()

,Year,Team,PA,OBP_calc,ISO_calc,BB_HBP_PA,SO_PA,HR_PA
0,2021,KIA,5577,0.337066,0.087560,0.119778,0.165143,0.011834
1,2021,KT,5626,0.356012,0.116216,0.124778,0.186100,0.018841
2,2021,LG,5533,0.341908,0.118316,0.123983,0.167359,0.019881
3,2021,NC,5530,0.342899,0.155431,0.112477,0.197649,0.030741
4,2021,SSG,5735,0.353521,0.160269,0.127289,0.183958,0.032258


In [7]:
# 득점 효율
team_year["R_PA"] = team_year["R"] / team_year["PA"].replace(0, np.nan)

# 비홈런 컨택 출루
team_year["H_minus_HR_PA"] = (team_year["H"] - team_year["HR"]) / team_year["PA"].replace(0, np.nan)

# 병살 리스크
team_year["GDP_PA"] = team_year["GDP"] / team_year["PA"].replace(0, np.nan)

team_year[["Year", "Team", "PA","R_PA", "H_minus_HR_PA", "GDP_PA"]].head()

,Year,Team,PA,R_PA,H_minus_HR_PA,GDP_PA
0,2021,KIA,5577,0.101847,0.202977,0.021338
1,2021,KT,5626,0.127800,0.207963,0.018663
2,2021,LG,5533,0.118200,0.194831,0.018435
3,2021,NC,5530,0.126944,0.196022,0.016998
4,2021,SSG,5735,0.131648,0.190584,0.018483


In [8]:
team_year.to_csv("../data/processed/team_year_batting_2021_2025.csv", index=False)

In [9]:
team_year.shape

(50, 30)

In [10]:
team_year["Team"].nunique()

10

---

In [11]:
pit = pit_season.copy()
pit["Year"] = pd.to_numeric(pit["Year"], errors="coerce")
pit = pit[pit["Year"].between(2021, 2025)]
pit = pit.dropna(subset=["Year", "Team"])

In [12]:
pit.columns

Index(['Id', 'Name', 'Birthdate', 'Handedness', 'School', 'Draft', 'Year',
       'Team', 'Age', 'Pos.', 'G', 'GS', 'GR', 'GF', 'CG', 'SHO', 'W', 'L',
       'S', 'HD', 'IP', 'ER', 'R', 'rRA', 'TBF', 'H', '2B', '3B', 'HR', 'BB',
       'HP', 'IB', 'SO', 'ROE', 'BK', 'WP', 'ERA', 'RA9', 'rRA9', 'FIP',
       'WHIP', 'WAR'],
      dtype='object')

In [13]:
team_year_pit = (
    pit.groupby(["Year","Team"], as_index=False)
       .agg({
           "IP":"sum",
           "H":"sum",
           "HR":"sum",
           "BB":"sum",
           "SO":"sum",
           "ER":"sum",
           "FIP":"mean",  # ✅ FIP는 rate stat이라 보통 mean이 자연스러움(표본 가중은 01에서 개선 가능)
       })
)

In [14]:
# 파생
team_year_pit["IP"] = team_year_pit["IP"].replace(0, np.nan)

team_year_pit["BB_9"] = team_year_pit["BB"] * 9 / team_year_pit["IP"]
team_year_pit["K_9"]  = team_year_pit["SO"] * 9 / team_year_pit["IP"]
team_year_pit["HR_9"] = team_year_pit["HR"] * 9 / team_year_pit["IP"]
team_year_pit["ERA"]  = team_year_pit["ER"] * 9 / team_year_pit["IP"]
team_year_pit["H_9"]  = team_year_pit["H"] * 9 / team_year_pit["IP"]

In [15]:
team_year_pit.to_csv("../data/processed/team_year_pitching_2021_2025.csv", index=False)

본 프로젝트에서는 팀 성과를  
공격 구조와 투수 구조의 결합 결과로 해석한다.

이에 따라 본 파일(00_build_team_year)에서는  
공격(team-year batting)과 투수(team-year pitching) 데이터를  
각각 독립적인 기초 레이어로 구축한다.

In [16]:
"FIP" in pit.columns

True